In [39]:
import pandas as pd
import time
import tqdm
import requests

In [40]:
DATA_DIRECTORY = '../../data/2023-01-08/'

In [41]:
df = pd.read_csv(f'{DATA_DIRECTORY}df_raw_data_rows.csv', index_col=0, usecols=[0,1,2,3,5,6,11,14])[[str(elem) for elem in [1,2,10,4,5,13,0]]]

In [42]:
df = df.rename(columns={"1":"Name", "2":"City", "10":"Street", "4":"Beds", "5":"Cases", "13":"Phone", "0":"Portraitlink"})

In [43]:
df.head()

,Name,City,Street,Beds,Cases,Phone,Portraitlink
0,Charité - Universitätsmedizin Berlin,10117 Berlin,Charitéplatz 1,3.011,1.510.450,030 45050,/app/portrait/28475032679be264/start
1,Universitätsmedizin der Johannes Gutenberg-Uni...,55131 Mainz,Langenbeckstraße 1,1.675,302.632,06131 17 0,/app/portrait/b985292fd83e0a2a/start
2,Universitätsklinikum Tübingen,72076 Tübingen,Geissweg 3,1.585,70.733,07071 29 0,/app/portrait/16d84a8154635564/start
3,Universitätsmedizin Göttingen,37075 Göttingen,Robert-Koch-Straße 40,1.563,274.225,0551 39 0,/app/portrait/7c0ee39a26fa8dec/start
4,Universitätsklinikum Freiburg,79106 Freiburg,Hugstetter Straße 55,1.554,561.570,0761 270 0,/app/portrait/bd490cfe8933289f/start


In [58]:
def get_coordinates(address):
    api_key = "YOUR_API_KEY"
    base_url = "https://nominatim.openstreetmap.org/search?"
    query_string = "format=json&limit=1&q="
    request_url = base_url + query_string + address
    response = requests.get(request_url)
    response_json = response.json()
    if len(response_json) > 0:
        lat = response_json[0]["lat"]
        lon = response_json[0]["lon"]
        return (lat, lon)
    else:
        return None

In [59]:
def get_by_address(row):
    try:
        query = f'{row["Street"]} {row["City"]}'
        result = get_coordinates(query)
        return result
    except:
        print("Fetch by address failed " + query)
        return false


In [60]:
def get_by_name(row):
    try:
        query = f'{row["Name"]}'
        result = get_coordinates(query)
        return result
    except:
        print("Fetch by name failed " + query)
        return False

In [61]:
def get_by_zip_code_and_street(row):
    try:
        query = f'{row["Street"]} {row["City"].strip()[:5]}'
        result = get_coordinates(query)
        return result
    except:
        print("Fetch by zip_code and street failed" + query)
        return false

In [62]:
def get_by_zip_code(row):
    try:
        query = f'{row["City"].strip()[:5]}'
        result = get_coordinates(query)
        return result
    except:
        print("Fetch by zip_code and failed" + query)
        return false

In [63]:
def append_result(result):
    coordinates.append([result[0], result[1]])

In [71]:
coordinates = []
for index, row in tqdm.tqdm(df.iterrows()):
    print(str(index) + "/" + str(len(df)))
    #time.sleep(1)
    result = get_by_address(row)
    if result:
        append_result(result)
        continue;
    
    result = get_by_name(row)
    if result:
        append_result(result)
        continue;
    
    result = get_by_zip_code_and_street(row)
    if result:
        append_result(result)
        continue;
    
    result = get_by_zip_code(row)
    if result:
        append_result(result)
        continue;
        
    print("Could not find lat and lon")
    coordinates.append([None, None])

0it [00:00, ?it/s]

0/2493


1it [00:00,  4.66it/s]

1/2493


2it [00:00,  2.26it/s]

...

9it [00:04,  1.98it/s]

9/2493


9it [00:04,  1.98it/s]

Fetch by address failed Carl-Neuberg-Straße 1 30625 Hannover


NameError: name 'false' is not defined

In [66]:
df[['Lat', 'Lon']] = coordinates

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2493 entries, 0 to 2492
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          2493 non-null   object 
 1   City          2493 non-null   object 
 2   Street        2493 non-null   object 
 3   Beds          2493 non-null   float64
 4   Cases         2493 non-null   object 
 5   Phone         2476 non-null   object 
 6   Portraitlink  2493 non-null   object 
 7   Lat           2493 non-null   object 
 8   Lon           2493 non-null   object 
dtypes: float64(1), object(8)
memory usage: 194.8+ KB


In [69]:
df_nan = df[(df['Lat'].isna() | df['Lon'].isna())]

In [70]:
df_nan.head()

,Name,City,Street,Beds,Cases,Phone,Portraitlink,Lat,Lon


In [ ]:
df_nan.to_csv(f'{DATA_DIRECTORY}nan_data.csv')